In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader, prep_config
from pure_funcs import denumpyize, numpyize, get_template_live_config, candidate_to_live_config
from procedures import make_get_ticks_cache, dump_live_config, load_live_config
from time import time
from passivbot import add_argparse_args
from analyze import analyze_fills
from njit_funcs import calc_bid_ask_thresholds
import sys
import argparse
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.optimize_config_path = 'configs/optimize/default.hjson'
        self.symbol = 'XMRUSDT'
        self.user = 'your_user_name'
        self.start_date = '2020-12-01'
        self.end_date = '2021-06-01'
optimize_config = await prep_config(Args())
live_config = get_template_live_config()
config = {**optimize_config, **live_config}
dl = Downloader(config)
sts = time()
ticks = await dl.get_ticks(True)
prices, buyer_maker, timestamps, emas, ratios = make_get_ticks_cache(optimize_config, ticks)
data = (prices, buyer_maker, timestamps, emas, ratios)
print(f'millis to load {len(ticks)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
df = pd.DataFrame({**{'price': data[0], 'buyer_maker': data[1], 'timestamp': data[2], 'ema': data[3]},
                   **{f'ratio_{i}': data[4][:,i] for i in range(len(data[4][0]))}})
df

In [ ]:
bytes_per_mb = 1000000
millis_per_day = 1000 * 60 * 60 * 24
sizebytes = 0
for d in data:
    print(d.dtype)
    sizebytes += sys.getsizeof(d)
bytes_per_tick = sizebytes / len(prices)
print('bytes_per_tick', bytes_per_tick, 'data size in mb', sizebytes  / bytes_per_mb)

n_days = (timestamps[-1] - timestamps[0]) / millis_per_day
ticks_per_day = len(prices) / n_days
bytes_per_day = bytes_per_tick * ticks_per_day
millis_per_tick = n_days / len(prices) * millis_per_day
print('n_days', n_days, 'millis_per_tick', millis_per_tick)

In [ ]:
config

In [ ]:
config['starting_balance'] = 100.0
config['latency_simulation_ms'] = 750
config['maker_fee'] = 0.00018

In [ ]:
tweaked = {
    'long': {
        'enabled': True,
        'stop_psize_pct': 0.1,
        'leverage': 3.0,
        'iqty_const': 0.01,
        'iprc_const': 0.985,
        'rqty_const': 0.5,
        'rprc_const': 0.98,
        'markup_const': 1.005,
        'iqty_MAr_coeffs': [
            [0.0, -0.0],
            [0.0, -0.0],
            [-0.0, -0.0]
        ],
        'iprc_MAr_coeffs': [
            [-0.0, -0.0],
            [0.0, -0.0],
            [0.0, 0.0]
        ],
        'rqty_MAr_coeffs': [
            [-0.0, 0.0],
            [-0.0, -0.0],
            [-0.0, 0.0]
        ],
        'rprc_MAr_coeffs': [
            [0.0, -0.0],
            [-0.0, 0.0],
            [0.0, 0.0]
        ],
        'rprc_PBr_coeffs': [
            [-0.0, 0.0]
        ],
        'markup_MAr_coeffs': [
            [0.0, -0.0],
            [-0.0, 0.0],
            [-0.0, 0.0]
        ]
    },
    'shrt': {
        'enabled': True,
        'stop_psize_pct': 0.1,
        'leverage': 3.0,
        'iqty_const': 0.01,
        'iprc_const': 1.015,
        'rqty_const': 0.5,
        'rprc_const': 1.02,
        'markup_const': 0.995,
        'iqty_MAr_coeffs': [
            [0.0, -0.0],
            [0.0, -0.0],
            [-0.0, -0.0]
        ],
        'iprc_MAr_coeffs': [
            [-0.0, -0.0],
            [0.0, -0.0],
            [0.0, 0.0]
        ],
        'rqty_MAr_coeffs': [
            [-0.0, 0.0],
            [-0.0, -0.0],
            [-0.0, 0.0]
        ],
        'rprc_MAr_coeffs': [
            [0.0, -0.0],
            [-0.0, 0.0],
            [0.0, 0.0]
        ],
        'rprc_PBr_coeffs': [
            [-0.0, 0.0]
        ],
        'markup_MAr_coeffs': [
            [0.0, -0.0],
            [-0.0, 0.0],
            [-0.0, 0.0]
        ]
    }
}

In [ ]:
config_to_test = {**config, **numpyize(tweaked)}
# dump_live_config(config_to_test, 'hand_tuned.json')

In [ ]:
sts = time()
fills, info = backtest(config_to_test, data)
print(f'seconds elapsed {time() - sts:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')

In [ ]:
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
fdf, analysis = analyze_fills(fills, result, timestamps[0], timestamps[-1])
fdf

In [ ]:
analysis

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()

In [ ]:
bids, asks = calc_bid_ask_thresholds(prices, emas, ratios, (config['long']['iprc_const'], config['shrt']['iprc_const']),
                                     (config['long']['iprc_MAr_coeffs'], config['shrt']['iprc_MAr_coeffs']))

In [ ]:
bidask = pd.DataFrame({'bid': bids, 'ask': asks})
bidask

In [ ]:
dfba = df[['price']].join(bidask)

In [ ]:
dfba.iloc[:3000000:100].plot()